In [2]:
import csv

torah = 'torah.csv'
megillot = 'chamesh megillot.csv'
navi1 = 'neviim rishonim.csv'
navi2 = 'neviim achronim.csv'
emet = 'sifrei emet.csv'
ketuvim = 'ketuvim.csv'


cant = ['֑','֒','֓','֔','֕','֖','֗','֘','֙','֚','֛','֜',
             '֝','֞','֠','֡','֢','֣','֤','֥','֦','֧','֨','֩','֪','֫','֬','֭','֯','׃']

kamatz_katan = "ׇ"

In [3]:
#removes trop but not vowels
def trop_remover(word):
    no_cant_word = ''
    for letter in word:
        if letter not in cant:
            no_cant_word = no_cant_word + letter
    return no_cant_word

In [34]:
def kamatz_katan_finder(bookset):
    words_with_katan = {"כׇּל":[],"כׇל":[]}
    with open(bookset, 'r', encoding='utf-8') as csvDataFile:
        csvReader = csv.reader(csvDataFile)
        in_range = False
        for row in csvReader:
            words = row[4].split(" ")
            for word in words:
                if kamatz_katan not in word:
                    continue
                elif trop_remover(word) not in words_with_katan.keys() and "־" not in word:
                    words_with_katan[trop_remover(word)] = [row[3]]
                elif trop_remover(word) in words_with_katan.keys() and "־" not in word:
                    words_with_katan[trop_remover(word)].append(row[3])
                elif "־" in word:
                    divided = word.split("־")
                    for piece in divided:
                        piece = trop_remover(piece)
                        if kamatz_katan in piece and "כׇּל" not in piece and "כׇל" not in piece:
                            if trop_remover(piece) not in words_with_katan:
                                words_with_katan[piece] = [row[3]]
                            else:
                                words_with_katan[trop_remover(piece)].append(row[3])
    return words_with_katan

In [44]:
 def exporter(katandict):
    with open("output.tex",'w',encoding='utf-8') as outfile:
        outfile.write("\\begin{itemize} ")
        for item in katandict:
            outfile.write("\item ")
            outfile.write(item)
            outfile.write("\n\\begin{itemize} ")
            for ref in katandict[item]:
                outfile.write("\item ")
                outfile.write(ref)
            outfile.write("\n\end{itemize}\n ")
        outfile.write("\end{itemize} ")

In [45]:
exporter(kamatz_katan_finder(torah))